<a href="https://colab.research.google.com/github/anmol-sinha-coder/DEmoClassi/blob/master/Age_Gender_Race_Emotion_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/G_Drive')

In [ ]:
! git clone https://github.com/anmol-sinha-coder/DEmoClassi.git
! cp -ra DEmoClassi/{vision_utils,emotion_detection,multitask_rag,'setup.py'} ./
! pip install tensorboardX pytorch-ignite pillow
! unzip /content/G_Drive/MyDrive/ADNN/facial-expression-recognition-challenge.zip -d .
! tar -xzvf /content/G_Drive/MyDrive/ADNN/UTKFace/UTKFace.tar.gz -C .
! tar -xzvf /content/G_Drive/MyDrive/ADNN/UTKFace/crop_part1.tar.gz -C .
! tar -xzvf fer2013.tar.gz
! cp /content/G_Drive/MyDrive/ADNN/cv2_gpu/cv2.cpython-36m-x86_64-linux-gnu.so .

In [ ]:
import torch
import torchvision.transforms as transforms

from vision_utils.custom_torch_utils import load_model
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model, PretrainedMT

from emotion_detection.evaluate import evaluate_model as eval_fer
from emotion_detection.fer_data_utils import *
from emotion_detection.train import run_fer

from multitask_rag.train import run_utk
from multitask_rag.utk_data_utils import get_utk_dataloader, split_utk
from multitask_rag.evaluate import evaluate_model as eval_utk
from multitask_rag.utk_data_utils import display_examples_utk

import numpy as np
import pandas as pd
import glob
import os
import tqdm
import random
import cv2
cv2.__version__

## Fer2013 dataset
Fer2013 is a kaggle dataset which consists of a set of 48x48 grayscale images representing the following facial expressions : 
* 0 : Angry
* 1 : Disgust
* 2 : Fear 
* 3 : Happy 
* 4 : Sad 
* 5 : Surprise 
* 6 : Neutral

In [ ]:
path_fer = './fer2013/fer2013.csv'
df_fer2013 = pd.read_csv(path_fer)

In [ ]:
display_examples_fer(df_fer2013, 0)

In [ ]:
display_examples_fer(df_fer2013, 1)

In [ ]:
display_examples_fer(df_fer2013, 2)

In [ ]:
display_examples_fer(df_fer2013, 3)

In [ ]:
display_examples_fer(df_fer2013, 4)

In [ ]:
display_examples_fer(df_fer2013, 5)

In [ ]:
display_examples_fer(df_fer2013, 6)

## UTKFace dataset
This is a dataset of cropped face images for the task of predicting the age, gender and race of a person.<br>

**Age :** A number between 0 and 101 (representing the age of the person)<br>

**Gender :**
* 0 : Male
* 1 : Female

**Race :**
* 0 : White
* 1 : Black
* 2 : Asian
* 3 : Indian
* 4 : Other


In [ ]:
path_utk = './UTKFace/'

In [ ]:
display_examples_utk(path_utk, 'gender', 0)

In [ ]:
display_examples_utk(path_utk, 'gender', 1)

In [ ]:
display_examples_utk(path_utk, 'race', 0)

In [ ]:
display_examples_utk(path_utk, 'race', 1)

In [ ]:
display_examples_utk(path_utk, 'age', 10)

# Training

Now that we have the data ready, let's move to the funniest part : model training!
As I have two separate datasets (`Fer2013` for emotion detection and `UTKFace` for gender-race-age prediction) we'll 
have to train two separate models. For each of the two tasks I tested 3 different architectures : 
* A CNN based on Depthwise Separable Convolution
* Finetuning a pretrained Resnet50
* Finetuning a pretrained VGG19

<hr  size=10 color=black> 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device, torch.cuda.is_available()

In [ ]:
DATA_DIR = "./fer2013/fer2013.csv" # path to the csv file
BATCH_SIZE = 256 # size of batches 
train_flag = 'Training' #`Usage` column in the csv file represents the usage of the data : train or validation or test
val_flag = 'PublicTest'


## (1)Training Emotion/Reaction detector
### [1.a] Depthwise Separable Convolution model
First we need to create DataLoader objects which are handy Pytorch objects for yielding batches of data during training.
Basically, what the following code does is : 
* read the csv file and convert the raw pixels into numpy arrays
* Apply some pre-processing operations : 
    * Histogram equalization
    * Add a channel dimension so that the image becomes 48x48x1 instead of 48x48
    * Convert the numpy array to a pytorch tensor

In [ ]:
# The transformations to apply
data_transforms = transforms.Compose([
    HistEq(), # Apply histogram equalization
    AddChannel(), # Add channel dimension to be able to apply convolutions
    transforms.ToTensor()
])

train_dataloader = get_fer_dataloader(BATCH_SIZE, DATA_DIR, train_flag, data_transforms=data_transforms)
validation_dataloader = get_fer_dataloader(BATCH_SIZE, DATA_DIR, val_flag, data_transforms=data_transforms)

my_data_loaders = {
    'train': train_dataloader,
    'valid': validation_dataloader
}

backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/Separable_Convolutional_Model'
os.makedirs(backup_path, exist_ok=True)  # create the directory if it doesn't exist
checkpoint = '/content/checkpoints/fer_sep_conv_histeq'  # folder where to save checkpoints during training
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
my_model = SepConvModel().to(device)
my_optimizer = torch.optim.Adam(my_model.parameters(), lr=1e-3)

In [ ]:
# Evaluation of model
run_fer(model=my_model, optimizer=my_optimizer, epochs=300,
        log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='FER-Sep_Conv',
        n_saved=1, log_dir=None, launch_tensorboard=False,
        patience=50, resume_model=None, resume_optimizer=None,
        backup_step=5, backup_path=backup_path,
        n_epochs_freeze=0, n_cycle=None)

### [1.b] Resnet-50
Over the years there is a trend to go more deeper, to solve more complex tasks and to also increase /improve the classification/recognition accuracy. But, as we go deeper; the training of neural network becomes difficult and also the accuracy starts saturating and then degrades also. Residual Learning tries to solve both these problems.

- In general, in a deep convolutional neural network, several layers are stacked and are trained to the task at hand.
- The network learns several low OR mid OR high level features at the end of its layers.
- In residual learning, instead of trying to learn some features, we try to learn some residual. Residual can be simply understood as subtraction of feature learned from input of that layer.
- ResNet does this using shortcut connections (directly connecting input of nth layer to some (n+x)th layer.
- It has proved that training this form of networks is easier than training simple deep convolutional neural networks and also the problem of degrading accuracy is resolved.

In [ ]:
# The transformations to 
data_transforms = transforms.Compose([
    HistEq(), # Apply histogram equalization
    ToRGB(),
    transforms.ToTensor()
])

my_data_loaders = {
    'train': get_fer_dataloader(BATCH_SIZE, DATA_DIR, train_flag, data_transforms=data_transforms),
    'valid': get_fer_dataloader(BATCH_SIZE, DATA_DIR, val_flag, data_transforms=data_transforms)
}

backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/ResNet_Model'
os.makedirs(backup_path, exist_ok=True)
checkpoint = '/content/checkpoints/fer_resnet_adam_histeq'
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
my_model, _ = initialize_model(model_name='resnet', feature_extract=True, num_classes=7, task='fer2013', use_pretrained=True, device=device)
# The optimizer must only track the parameters that are trainable (thus excluding frozen ones)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3) 

In [ ]:
run_fer(model=my_model, optimizer=my_optimizer, epochs=200,
        log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='FER-Resnet',
        n_saved=1, log_dir=None, launch_tensorboard=False,
        patience=75, val_monitor='acc', resume_model=None,
        resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=10, n_cycle=None)

### [1.c] VGG-19
VGG19 is a variant of VGG model which in short consists of 19 layers (16 convolution layers, 3 Fully connected layer, 5 MaxPool layers and 1 SoftMax layer).

- Conv3x3 (64)
- Conv3x3 (64)
- MaxPool
- Conv3x3 (128)
- Conv3x3 (128)
- MaxPool
- Conv3x3 (256)
- Conv3x3 (256)
- Conv3x3 (256)
- Conv3x3 (256)
- MaxPool
- Conv3x3 (512)
- Conv3x3 (512)
- Conv3x3 (512)
- Conv3x3 (512)
- MaxPool
- Conv3x3 (512)
- Conv3x3 (512)
- Conv3x3 (512)
- Conv3x3 (512)
- MaxPool
- Fully Connected (4096)
- Fully Connected (4096)
- Fully Connected (1000)
- SoftMax


In [ ]:
# The transformations to 
data_transforms = transforms.Compose([
    HistEq(),
    ToRGB(), 
    transforms.ToTensor()
])

my_data_loaders = {
    'train': get_fer_dataloader(256, DATA_DIR, train_flag, data_transforms=data_transforms),
    'valid': get_fer_dataloader(512, DATA_DIR, val_flag, data_transforms=data_transforms)
}

my_model, _ = initialize_model(model_name='vgg', feature_extract=True, num_classes=7, task='fer2013', use_pretrained=True, device=device)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3)

backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/VGGNet_Model'
os.makedirs(backup_path, exist_ok=True)
checkpoint = '/content/checkpoints/fer_vgg_adam_histeq'
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
run_fer(model=my_model, optimizer=my_optimizer, epochs=300,
        log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='FER-VGGnet',
        n_saved=1, log_dir=None, launch_tensorboard=False,
        patience=100, val_monitor='acc',
        resume_model=None, resume_optimizer=None,
        backup_step=5, backup_path=backup_path,
        n_epochs_freeze=20, n_cycle=None)

<hr>

# (2)Training Age, Gender, Race detector

In [ ]:
! mkdir logs/
list_images = glob.glob('/content/UTKFace/*jp*')
print(len(list_images))

In [ ]:
# Labels are given in the image names. the image names format is the following : age_gender_race_date.
# for instance this image name 1_0_0_20161219140623097.jpg.chip.jpg suggests that the image corresponds
# to a person whose age is 1, gender is 0 (Male) and race is 0 (White). However there are few images for 
# which the name is malfomed, so we remove them using the following code snippet :
# function to remove invalid images (that he filenames is not correctly formatted)
def get_invalid_images(root_path='/content/data/UTKFace/'):
    list_files = glob.glob(os.path.join(root_path, '*.[jJ][pP]*'))
    filenames = [path.split('/')[-1].split('_') for path in list_files]
    print()
    invalid_images = []
    for i, im in enumerate(tqdm.tqdm(filenames)):
        if im[0].isdigit() and im[1].isdigit() and im[2].isdigit():
            continue
        else:
            invalid_images.append(list_files[i])
    return invalid_images


invalid_images = get_invalid_images()
print(invalid_images)
for f in invalid_images: # removal of invalid images
    os.remove(f)

## [2.a] Depthwise Separable Convolution model

In [ ]:
# split the dataset into train, test and validation sets 
SRC_DIR = '/content/UTKFace/'  # path to the folder containing all images
DEST_DIR = '/content/utk_split/' # path where to save the split dataset, 3 subdirectories will be created (train, valid and test)
SPLIT = 0.7 # ratio of the train set, the remaining (30%) will be split equally between validation and test sets
split_utk(SRC_DIR, DEST_DIR, SPLIT)

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_loader = get_utk_dataloader(batch_size=128, data_dir=DEST_DIR, data_transforms=data_transforms, flag='train')
val_loader = get_utk_dataloader(batch_size=128, data_dir=DEST_DIR, data_transforms=data_transforms, flag='valid')

my_data_loaders = {
    'train': train_loader,
    'valid': val_loader
}

backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/Separable_Convolutional_Model'
os.makedirs(backup_path, exist_ok=True)
checkpoint = '/content/checkpoints/utk_sep_conv_histeq'  # folder where to save checkpoints during training
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
my_model = SepConvModelMT(dropout=0.7, n_class=[1, 2, 5], n_filters=[64, 128, 256, 512], kernels_size=[3, 3, 3, 3]).to(device)
my_optimizer = torch.optim.Adam(my_model.parameters(), lr=1e-3)

In [ ]:
run_utk(my_model, my_optimizer, epochs=300,
        log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='UTK-Sep_Conv',
        n_saved=1, log_dir='/content/logs', launch_tensorboard=False,
        patience=50, resume_model=None, resume_optimizer=None,
        backup_step=5, backup_path=backup_path,
        n_epochs_freeze=0, lr_after_freeze=None,
        loss_weights=[1/10, 1/0.16, 1/0.44])

## [2.b] ResNet-50 model

In [ ]:
backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/ResNet_Model'
os.makedirs(backup_path, exist_ok=True)
checkpoint = '/content/checkpoints/utk_resnet_adam_histeq'
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
my_model = PretrainedMT(model_name='resnet', feature_extract=True, use_pretrained=True).to(device)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3)

In [ ]:
run_utk(my_model, my_optimizer, epochs=300, log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='UTK-Resnet', n_saved=1,
        log_dir='/content/logs', launch_tensorboard=False, patience=50,
        resume_model=None, resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=10, n_cycle=None, lr_after_freeze=1e-4,
        loss_weights=[1/10, 1/0.16, 1/0.44], lr_plot=True)

## [2.c] VGG-19 model

In [ ]:
backup_path = '/content/G_Drive/MyDrive/ADNN/Demography_Psychology/VGGNet_Model'
os.makedirs(backup_path, exist_ok=True)
checkpoint = '/content/checkpoints/utk_vgg_adam_histeq'
os.makedirs(checkpoint, exist_ok=True)

In [ ]:
my_model = PretrainedMT(model_name='vgg', feature_extract=True, use_pretrained=True).to(device)
my_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1e-3)

In [ ]:
run_utk(my_model, my_optimizer, epochs=300, log_interval=1, dataloaders=my_data_loaders,
        dirname=checkpoint, filename_prefix='UTK-VGGnet', n_saved=1,
        log_dir='/content/logs', launch_tensorboard=False, patience=50,
        resume_model=None, resume_optimizer=None, backup_step=5, backup_path=backup_path,
        n_epochs_freeze=10, n_cycle=None, lr_after_freeze=1e-4,
        loss_weights=[1/10, 1/0.16, 1/0.44], lr_plot=True)